# <span style="color:brown"> **Hands-on Deep Learning – Image Classification**

_This hands-on was originally created by Thomas Grenier (TensorFlow) and Fabien Millioz (PyTorch), CREATIS.

thomas.grenier@creatis.insa-lyon.fr,
michael.sdika@creatis.insa-lyon.fr,
olivier.bernard@creatis.insa-lyon.fr,
odyssee.merveille@creatis.insa-lyon.fr,
fabien.millioz@creatis.insa-lyon.fr

# <span style="color:brown"> **Introduction on data**

This hands-on is computer scientist-oriented more than application-oriented.

And so, this practice focuses on a meaningless toy example inspired from the MNIST manuscript numbers classification challenge that is considered as the 'hello world' example for some machine learning and almost all neural networks.

MR images come from [IXI Dataset](https://brain-development.org/ixi-dataset/)

Here, we have to recognize whether a brain slice image is axial, sagittal or coronal and comes from MRI T1w, MRI T2w and Proton Density MR (PD). So, this is a classification problem with 9 classes, as follows:
   
<img src="./figures/Figure1_SagittalAxialCoronal_small.png" alt="SagittalAxialCoronal" style="width: 60%"/>

Amount of images of each class for train and test datasets is :  
- Total images for training:
   - PD-A (PD Axial)    : 2701
   - PD-C (PD Coronal)  : 2723
   - PD-S (PD Sagittal) : 2698
   - T1-A (T1 Axial)    : 2715
   - T1-C (T1 Coronal)  : 2758
   - T1-S (T1 Sagittal) : 2711
   - T2-A (T2 Axial)    : 2696
   - T2-C (T2 Coronal)  : 2717
   - T2-S (T2 Sagittal) : 2699
- Total images for testing:
   - PD-A (PD Axial)     : 689
   - PD-C (PD Coronal)   : 696
   - PD-S (PD Sagittal)  : 693
   - T1-A (T1 Axial)     : 679
   - T1-C (T1 Coronal)   : 685
   - T1-S (T1 Sagittal)  : 676
   - T2-A (T2 Axial)     : 684
   - T2-C (T2 Coronal)   : 690
   - T2-S (T2 Sagittal)  : 686

We have enough data to test classification approaches such as random forest, MLP and CNN.

> Let's go!



# <span style="color:brown"> **A - Setup**

### A1 - First, import a few common modules, ensure MatplotLib plots figures inline, and functions for converting data and displaying reports:

In [ ]:
#!pip install --upgrade pip

# Common imports
import os
import glob

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import numpy as np

!pip install tqdm
from tqdm import tqdm

!pip install scikit-learn
from sklearn import datasets, metrics

# if trouble with libGL consider to execute this line in a shell: !apt-get install python3-opencv
#!pip install opencv-contrib-python-headless
#import cv2


### A2- We import TensorFlow 2

In [ ]:
import tensorflow as tf
print(tf.__version__)

> _**If no error occurs, ie the version of TensorFlow is printed, your working environment is ok and you can go to the next part,
> else ... call an assistant for help!**_

# <span style="color:brown"> **B - Read the data**

<span style="color:red">
    

We first download the data and then deflate them.

In [ ]:
!mkdir -p data
#if not os.path.isfile("./data/dlss_classification_A.tar.gz"):
    
!wget -P data "https://www.creatis.insa-lyon.fr/~grenier/wp-content/uploads/teaching/DeepLearning/dlss_classification_A.tar.gz"
!tar xzf ./data/dlss_classification_A.tar.gz -C data
#else:
#    print(" ** file dlss_classification_A.tar.gz already downloaded, skipping downloading and untar \n Remove the file dlss21_ho1_data.tar.gz to download it again. \n This is not an error :) " )

### <span style="color:brown"> **B1 - Open files and convert to numpy / tensorflow** 

Now start by reading _png_ files stored in the data train and test directories. 
Note that the filename of the _png_ image describes which view (Axial, Coronal, Sagital) and modality (T1w, T2w and PD) correspond to the image.

These _png_ correspond to slices extracted from 3D volumes of the [IXI dataset](https://brain-development.org/ixi-dataset/). In order to reduce the data variability (and simplify the slice extraction scripts), all MRI volumes were registered on a reference image using [elastix](http://elastix.isi.uu.nl/) with an affine transform. 

Thus, origin, spacing, orientation and number of pixels per dimension of all volumes are the same and therefore will not need to be taken into account systematically by the learning scripts (using SimpleITK, MedPy, nibabel...).

Then, slices are extracted and the window/level was adapted (using [ITK](https://itk.org/)).

Finally, they were resized then padded/cropped to 64x64 using [ImageMagick](https://www.imagemagick.org/).

Image files in __data/dlss_classification_A/train/__  are organized as follow:
  - dlss_classification_A/train/PD-A/002_116_PD-A.png
                              /002_42_PD-A.png
                              /002_44_PD-A.png
                              /002_53_PD-A.png
                               ...
  - dlss_classification_A/train/PD-C/002_152_PD-C.png
                              /002_175_PD-C.png
                              /002_177_PD-C.png
                              /002_198_PD-C.png
                               ...
  - data/dlss_classification_A/train/PD-S/...
  - data/dlss_classification_A/train/T1-A/...
  - data/dlss_classification_A/train/T1-C/...
  - data/dlss_classification_A/train/T1-S/...
  - data/dlss_classification_A/train/T2-A/...
  - data/dlss_classification_A/train/T2-C/...
  - data/dlss_classification_A/train/T2-S/...

The same is true for __data/dlss_classification_A/test/__ with different subjects than the ones present in the train set.


> The next cells will open all images and create three datasets containing the images and their corresponding labels.


Some important values and initializations:

In [ ]:
desired_height = 64
desired_width  = 64

input_train_path = os.getcwd()+'/data/dlss_classification_A/train'           
input_test_path = os.getcwd()+'/data/dlss_classification_A/test'            
print(f"input train path : {input_train_path}")
print(f"input test path  : {input_test_path}")

class_names = ['PD-A', 'PD-C', 'PD-S', 'T1-A', 'T1-C', 'T1-S', 'T2-A', 'T2-C', 'T2-S']

# Definition of constants
input_shape = (desired_height,desired_width,1)
input_shape_rgb = (desired_height,desired_width,3)
num_classes = 9

In [ ]:
# first list files for the train and validation sets.
train_val_image_file_list = []
train_val_label_list = []

for i in range(len(class_names)):
    for filename in glob.iglob( os.path.join(input_train_path,class_names[i]) + '/**/*.png', recursive=True):
        # extract patient number and slice
        train_val_image_file_list.append(filename)
        train_val_label_list.append( i )     

print(len(train_val_image_file_list), len(train_val_label_list))
print(train_val_image_file_list[0])

In [ ]:
# Random permutation
permutation = np.random.permutation( len(train_val_label_list) )
train_val_images_files=[train_val_image_file_list[i] for i in permutation]
train_val_labels=[train_val_label_list[i] for i in permutation]

In [ ]:
# we split the train set in two parts : train and validation
VALIDATION_RATIO = 0.2
nb_train = int( len(train_val_label_list) * (1 - VALIDATION_RATIO) )
# list images for both datasets
train_images_files = train_val_images_files[:nb_train]
train_labels =train_val_labels[:nb_train]

val_images_files = train_val_images_files[nb_train:]
val_labels = train_val_labels[nb_train:]

# for labels also convert them to numpy
y_train = np.asarray(train_labels)
y_val = np.asarray(val_labels)

print( train_val_images_files[0])

In [ ]:
from PIL import Image, ImageFilter
# Reading images 
def ReadImages(images_files):
    X = []
    for index in tqdm(range(len(images_files))):
        image_read = Image.open(images_files[index])
        image_read = image_read.resize((desired_width, desired_height), resample = Image.Resampling.BICUBIC)
        image_read = np.array(image_read)
        image_read_rgb = np.expand_dims(image_read,-1)
        image_read_rgb = np.repeat(image_read_rgb, 3, axis=-1)
        X.append(image_read_rgb )
    X = np.asarray(X, dtype=np.uint8)
    return X

In [ ]:
# Read images
X_train_rgb = ReadImages(train_images_files)
X_val_rgb   = ReadImages(val_images_files)

In [ ]:
image_read = Image.open(train_images_files[0])
image_read = image_read.resize((desired_width, desired_height), resample = Image.Resampling.BICUBIC)
image_read = np.array(image_read)
image_read_rgb = np.expand_dims(image_read,-1)
image_read_rgb = np.repeat(image_read_rgb, 3, axis=-1)
print(image_read_rgb.shape)

### <span style="color:brown"> **B2 - Check the output data size and type**
We recommend to check as often as needed the shape and type of data.

Don't go further if you can't see what is inside array and tensor!

In [ ]:
# Show information
print(" Shape : " , X_train_rgb.shape, X_val_rgb.shape)
print(" Type  : %s  %s"%(X_train_rgb.dtype, X_val_rgb.dtype))
print(" Max   : %d  %d"%(X_train_rgb.max(), X_val_rgb.max()))
print(" Min   : %d  %d"%(X_train_rgb.min(), X_val_rgb.min()))

print(" Labels Shape : " , y_train.shape, y_val.shape)

### <span style="color:brown"> **B3 - Displaying images**

In [ ]:
def plot_image(image):
    plt.imshow(image, aspect="equal", cmap="gray", interpolation="nearest")
    plt.axis("off")    

In [ ]:
nb_x = 5
nb_y = 5
plt.figure(figsize=(15,15*nb_x/nb_y), dpi=100)
for i in range(1, nb_x * nb_y +1):
    plt.subplot(nb_x, nb_y ,i)
    plot_image( X_train_rgb[i-1] )
    plt.title(class_names[y_train[i-1]])

### <span style="color:brown"> **B4 - Test set**

In [ ]:
# doing the same for test set
test_image_file_list = []
test_label_list = []
           
for i in range(len(class_names)):
    for filename in glob.iglob( os.path.join(input_test_path,class_names[i]) + '/**/*.png', recursive=True):
        # extract patient number and slice
        test_image_file_list.append(filename)
        test_label_list.append( i )            

print(len(test_image_file_list), len(test_label_list))

X_test_rgb = ReadImages(test_image_file_list)
y_test = np.asarray(test_label_list)

print(" Shape : " , X_test_rgb.shape, y_test.shape)
print(" Type  : %s  %s"%(X_test_rgb.dtype, y_test.dtype))
print(" Max   : %d  %d"%(X_test_rgb.max(), y_test.max()))
print(" Min   : %d  %d"%(X_test_rgb.min(), y_test.min()))

nb_x = 5
nb_y = 5
plt.figure(figsize=(15,15*nb_x/nb_y), dpi=100)
for i in range(1, nb_x * nb_y +1):
    plt.subplot(nb_x, nb_y ,i)
    plot_image( X_test_rgb[i-1] )
    plt.title(class_names[y_test[i-1]])
# here we have not shuffle filenames :p

Verify shapes that should look like ( batch_size, desired_height, desired_width, 3 ) and (batch_size, )

### <span style="color:brown"> **B5 - Convert the data shape and type**
The _png_ files are RGB, we convert them to gray images for each set (train, validation and test) in order to be compatible with all tested approaches.

We also divide intensity values by 255 to bound range in [0;1]


In [ ]:
X_train_rgb.shape

In [ ]:
# converting the three data set : rgb to gray scale images, then normalization

X_train = X_train_rgb[:,:,:,1].astype(np.float32)/255
X_train = np.expand_dims(X_train,-1)

X_val = X_val_rgb[:,:,:,1].astype(np.float32)/255
X_val = np.expand_dims(X_val,-1)

X_test = X_test_rgb[:,:,:,1].astype(np.float32)/255
X_test = np.expand_dims(X_test,-1)

print(" Shape : ", X_train.shape, X_val.shape, X_test.shape)
print(" Type  : %s  %s %s"%(X_train.dtype, X_val.dtype, X_test.dtype))
print(" Max   : %d %d %d"%(X_train.max(), X_val.max(), X_test.max()) )
print(" Min   : %d %d %d"%(X_train.min(),X_val.min(), X_test.min()) )
plot_image( X_test[0].squeeze() )

An other way of representing labels is the one hot encoding form. MAny mathematical function can easly be generalized using this notation.
If they are 4 different labels, the sparse notation becomes:
   - label '0' --> 1 0 0 0
   - label '1' --> 0 1 0 0
   - label '2' --> 0 0 1 0
   - label '3' --> 0 0 0 1

This conversion can be done using a dedicated function:

In [ ]:
y_train_cat = tf.keras.utils.to_categorical(y_train)
y_val_cat = tf.keras.utils.to_categorical(y_val)
y_test_cat = tf.keras.utils.to_categorical(y_test)

print(" Original (sparse) shape  : ", y_train.shape, y_val.shape, y_test.shape)
print(" Categorical shape  : ", y_train_cat.shape, y_val_cat.shape, y_test_cat.shape)
print(" Example : ", y_train[0,], " --> " ,y_train_cat[0,])

# Now changing name :
y_train = y_train_cat
y_val = y_val_cat
y_test = y_test_cat

### <span style="color:red"> **Question**
- What means the 9 for the last axis of y_train_cat, y_val_cat, y_test_cat?
    

> _**All done, go to the classification step!**_

### <span style="color:brown"> **Now : 1- shutdown the kernel**
(menu --> Kernel --> "Shut down kernel")

### <span style="color:brown"> **Now : 2- go to next notebook!**
02_Classification_TF22